<b>1:  Import packages </b>

In [14]:
import pandas as pd
import numpy as np

<b>2: Import the project data </b>

In [116]:
dataset = pd.read_sas('C:/Users/u86004/Documents/Documentos Miguel/Pessoal/Mestrado/Materiais/Data Mining/Project/data/a2z_insurance.sas7bdat')

In [117]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10296 entries, 0 to 10295
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CustID         10296 non-null  float64
 1   FirstPolYear   10266 non-null  float64
 2   BirthYear      10279 non-null  float64
 3   EducDeg        10279 non-null  object 
 4   MonthSal       10260 non-null  float64
 5   GeoLivArea     10295 non-null  float64
 6   Children       10275 non-null  float64
 7   CustMonVal     10296 non-null  float64
 8   ClaimsRate     10296 non-null  float64
 9   PremMotor      10262 non-null  float64
 10  PremHousehold  10296 non-null  float64
 11  PremHealth     10253 non-null  float64
 12  PremLife       10192 non-null  float64
 13  PremWork       10210 non-null  float64
dtypes: float64(13), object(1)
memory usage: 1.1+ MB


In [118]:
dataset.head()

,CustID,FirstPolYear,BirthYear,EducDeg,MonthSal,GeoLivArea,Children,CustMonVal,ClaimsRate,PremMotor,PremHousehold,PremHealth,PremLife,PremWork
0,1.0,1985.0,1982.0,b'2 - High School',2177.0,1.0,1.0,380.97,0.39,375.85,79.45,146.36,47.01,16.89
1,2.0,1981.0,1995.0,b'2 - High School',677.0,4.0,1.0,-131.13,1.12,77.46,416.20,116.69,194.48,106.13
2,3.0,1991.0,1970.0,b'1 - Basic',2277.0,3.0,0.0,504.67,0.28,206.15,224.50,124.58,86.35,99.02
3,4.0,1990.0,1981.0,b'3 - BSc/MSc',1099.0,4.0,1.0,-16.99,0.99,182.48,43.35,311.17,35.34,28.34
4,5.0,1986.0,1973.0,b'3 - BSc/MSc',1763.0,4.0,1.0,35.23,0.90,338.62,47.80,182.59,18.78,41.45


In [119]:
dataset.shape

(10296, 14)

In [120]:
dataset.isna().sum()

CustID             0
FirstPolYear      30
BirthYear         17
EducDeg           17
MonthSal          36
GeoLivArea         1
Children          21
CustMonVal         0
ClaimsRate         0
PremMotor         34
PremHousehold      0
PremHealth        43
PremLife         104
PremWork          86
dtype: int64

In [121]:
dataset.shape

(10296, 14)

<b> Check if we have missing data </b>

In the data we can find that we have in total 309 rows with missing data. First I will impute the ones with the premiums to 0, so what I'm saying is that if they dont have premiums in these products the premium is 0. The other rows Im going to remove.

In [122]:
## imputing product premiums

dataset['PremMotor'].fillna(0, inplace=True)
dataset['PremHealth'].fillna(0, inplace=True)
dataset['PremLife'].fillna(0, inplace=True)
dataset['PremWork'].fillna(0, inplace=True)

In [123]:
dataset.isna().sum()

CustID            0
FirstPolYear     30
BirthYear        17
EducDeg          17
MonthSal         36
GeoLivArea        1
Children         21
CustMonVal        0
ClaimsRate        0
PremMotor         0
PremHousehold     0
PremHealth        0
PremLife          0
PremWork          0
dtype: int64

In [124]:
dataset.dropna(inplace=True)

In [125]:
dataset.shape

(10204, 14)

In [126]:
print('we kept {:.2f} of the data'.format((10204/10296)*100))

we kept 99.11 of the data


<b> Creating new variables </b>

In [141]:
## create new variables for client and policy tenure

## client tenure
dataset['ClientTenure'] = 2016-dataset['FirstPolYear']

## policy tenure
dataset['Policytenure'] = 2016-dataset['BirthYear'] #check this with the teacher

## client has specific LoB
dataset['HasMotor'] = np.where(dataset['PremMotor']>0,1,0)
dataset['HasHome'] = np.where(dataset['PremHousehold']>0,1,0)
dataset['HasHealth'] = np.where(dataset['PremHealth']>0,1,0)
dataset['HasLife'] = np.where(dataset['PremLife']>0,1,0)
dataset['HasWork'] = np.where(dataset['PremWork']>0,1,0)

## total number of LoB
dataset['NumberProducts'] = dataset['HasMotor']+dataset['HasHome']+dataset['HasHealth']+dataset['HasLife']+dataset['HasWork']

## is the client multi-product

dataset['IsEquiped'] = np.where(dataset['NumberProducts']>1,1,0)

##

dataset['PremMotor2'] = np.where(dataset['PremMotor']>0,dataset['PremMotor'],0)
dataset['PremHousehold2'] = np.where(dataset['PremHousehold']>0,dataset['PremMotor'],0)
dataset['PremHealth2'] = np.where(dataset['PremMotor']>0,dataset['PremMotor'],0)
dataset['PremLife2'] = np.where(dataset['PremMotor']>0,dataset['PremMotor'],0)

## total premiums

dataset['TotalPremiums'] = dataset['PremMotor']+dataset['PremHousehold']+dataset['PremHealth']+dataset['PremLife']+dataset['PremWork']

## earn premiums

dataset['EarnPremiums'] = dataset['PremMotor']+dataset['PremHousehold']+dataset['PremHealth']+dataset['PremLife']+dataset['PremWork']

## average client premium by LoB

dataset['AveragePremium'] = dataset['TotalPremiums']/dataset['NumberProducts']

## client margin

dataset['ClientPercMargin'] = 1-dataset['ClaimsRate']
dataset['ClientMargin'] = dataset['ClientPercMargin']*dataset['TotalPremiums']

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [128]:
## encoding level of education
dataset['EducationLevel'] = np.where(dataset['EducDeg'] == b'1 - Basic', 1,
                                    np.where(dataset['EducDeg'] == b'2 - High School', 2,
                                             np.where(dataset['EducDeg'] == b'3 - BSc/MSc', 3, 4)))

In [129]:
## geographic area

# changing the type to int to be a more likely name
dataset['GeoLivArea'] = dataset['GeoLivArea'].astype(int)

# create dummies
living_dummies = pd.get_dummies(dataset['GeoLivArea'], prefix = 'GeoLiv')

In [100]:
living_dummies

,GeoLiv_1,GeoLiv_2,GeoLiv_3,GeoLiv_4
0,1,0,0,0
1,0,0,0,1
2,0,0,1,0
3,0,0,0,1
4,0,0,0,1
...,...,...,...,...
10291,0,1,0,0
10292,0,0,1,0
10293,1,0,0,0
10294,0,1,0,0


In [130]:
## concatenate these dummies to the dataset

dataset = pd.concat([dataset,living_dummies], axis=1)

In [131]:
dataset.head()

,CustID,FirstPolYear,BirthYear,EducDeg,MonthSal,GeoLivArea,Children,CustMonVal,ClaimsRate,PremMotor,...,IsEquiped,TotalPremiums,AveragePremium,ClientPercMargin,ClientMargin,EducationLevel,GeoLiv_1,GeoLiv_2,GeoLiv_3,GeoLiv_4
0,1.0,1985.0,1982.0,b'2 - High School',2177.0,1,1.0,380.97,0.39,375.85,...,1,665.56,133.112,0.61,405.9916,2,1,0,0,0
1,2.0,1981.0,1995.0,b'2 - High School',677.0,4,1.0,-131.13,1.12,77.46,...,1,910.96,182.192,-0.12,-109.3152,2,0,0,0,1
2,3.0,1991.0,1970.0,b'1 - Basic',2277.0,3,0.0,504.67,0.28,206.15,...,1,740.60,148.120,0.72,533.2320,1,0,0,1,0
3,4.0,1990.0,1981.0,b'3 - BSc/MSc',1099.0,4,1.0,-16.99,0.99,182.48,...,1,600.68,120.136,0.01,6.0068,3,0,0,0,1
4,5.0,1986.0,1973.0,b'3 - BSc/MSc',1763.0,4,1.0,35.23,0.90,338.62,...,1,629.24,125.848,0.10,62.9240,3,0,0,0,1


In [136]:
dataset.loc[dataset['CustID']==1,['PremMotor','PremHousehold','PremHealth','PremLife','PremWork','NumberProducts','TotalPremiums','ClaimsRate']]

,PremMotor,PremHousehold,PremHealth,PremLife,PremWork,NumberProducts,TotalPremiums,ClaimsRate
0,375.85,79.45,146.36,47.01,16.89,5,665.56,0.39


In [139]:
dataset.IsEquiped.value_counts().sort_index()

0       13
1    10191
Name: IsEquiped, dtype: int64

In [135]:
dataset.NumberProducts.value_counts().sort_index() 

## existem 12 clientes sem produtos
## existe apenas 1 cliente com 1 produto
## maior parte do dataset tem os 5 produtos


0      12
1       1
2      59
3     410
4    1910
5    7812
Name: NumberProducts, dtype: int64

Clientes sem produtos

In [138]:
dataset.loc[dataset['NumberProducts']==0, ['CustID','PremMotor','PremHousehold','PremHealth','PremLife','PremWork','NumberProducts','TotalPremiums','ClaimsRate']]

,CustID,PremMotor,PremHousehold,PremHealth,PremLife,PremWork,NumberProducts,TotalPremiums,ClaimsRate
862,863.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
1133,1134.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
3165,3166.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
4022,4023.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
4113,4114.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
4271,4272.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
5983,5984.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
6439,6440.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
6614,6615.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
8585,8586.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0


In [140]:
dataset.loc[dataset['NumberProducts']==1, ['CustID','PremMotor','PremHousehold','PremHealth','PremLife','PremWork','NumberProducts','TotalPremiums','ClaimsRate']]

,CustID,PremMotor,PremHousehold,PremHealth,PremLife,PremWork,NumberProducts,TotalPremiums,ClaimsRate
2958,2959.0,390.52,-30.55,0.0,-1.11,-0.11,1,358.75,1.17


In [ ]:
dataset.drop(columns=['FirstPolYear','BirthYear','EducDeg','GeoLivArea'])